In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ROS']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('reactive oxygen species', 64016),
 ('reactive oxygen', 231),
 ('rod outer segments', 205),
 ('reactive oxidative stress', 39),
 ('reactive species of oxygen', 24),
 ('rosiglitazone', 18),
 ('reactive oxygen species production', 17),
 ('reactive oxygen spices', 15),
 ('rosuvastatin', 15),
 ('reactive oxygen species generation', 15),
 ('robot operating system', 9),
 ('reactive oxygen species formation', 9),
 ('rats osteosarcoma', 8),
 ('reactive oxygen species levels', 8),
 ('increased oxidative stress', 7),
 ('reactive oxygen substances', 6),
 ('reactive oxygenspecies', 6),
 ('reactive oxygen intermediates', 6),
 ('reactive oxygen specious', 5),
 ('reactive species oxygen', 5),
 ('ctrl or h2o2 on two successfully days', 5),
 ('reactive oxygen species assay', 4),
 ('rod photoreceptor outer segments', 4),
 ('reactive oxygen specified', 3),
 ('region of support', 3),
 ('reactive oxygen speices', 3),
 ('roscovitine', 3),
 ('reactive oxygen stress', 3),
 ('regression on order statisticall

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'CHEBI:CHEBI:26523': 'reactive oxygen species',
 'CHEBI:CHEBI:15379': 'dioxygen',
 'GO:GO:0120200': 'rod photoreceptor outer segment',
 'MESH:D018384': 'Oxidative Stress',
 'CHEBI:CHEBI:50122': 'rosiglitazone',
 'MESH:D017365': 'Spices',
 'CHEBI:CHEBI:38545': 'rosuvastatin',
 'GO:GO:1903409': 'reactive oxygen species biosynthetic process',
 'MESH:D012516': 'Osteosarcoma',
 'CHEBI:CHEBI:45307': 'seliciclib',
 'CHEBI:CHEBI:50699': 'oligosaccharide',
 'HGNC:10261': 'ROS1',
 'MESH:D012374': 'Rod Outer Segment',
 'MESH:D017382': 'Reactive Oxygen Species'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-03 04:45:43] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl
ERROR: [2020-11-03 04:47:47] adeft.gui.ground - Exception on /ground_add [POST]
Traceback (most recent call last):
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/adeft/.virtualenvs/adeft_env/lib/python3.8/site-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  F

In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'ctrl or h2o2 on two successfully days': 'ungrounded',
  'increased oxidative stress': 'MESH:D018384',
  'rats osteosarcoma': 'MESH:D012516',
  'reactive oxidative stress': 'MESH:D018384',
  'reactive oxygen': 'MESH:D017382',
  'reactive oxygen intermediates': 'MESH:D017382',
  'reactive oxygen species': 'MESH:D017382',
  'reactive oxygen species assay': 'MESH:D017382',
  'reactive oxygen species formation': 'GO:GO:1903409',
  'reactive oxygen species generation': 'GO:GO:1903409',
  'reactive oxygen species levels': 'MESH:D017382',
  'reactive oxygen species production': 'GO:GO:1903409',
  'reactive oxygen specified': 'MESH:D017382',
  'reactive oxygen specious': 'MESH:D017382',
  'reactive oxygen speices': 'MESH:D017382',
  'reactive oxygen spices': 'MESH:D017382',
  'reactive oxygen stress': 'MESH:D018384',
  'reactive oxygen substances': 'MESH:D017382',
  'reactive oxygenspecies': 'MESH:D017382',
  'reactive species of oxygen': 'MESH:D017382',
  'reactive species oxygen': 'MESH:D0

In [11]:
grounding_map, names, pos_labels = [{'ctrl or h2o2 on two successfully days': 'ungrounded',
  'increased oxidative stress': 'MESH:D018384',
  'rats osteosarcoma': 'MESH:D012516',
  'reactive oxidative stress': 'MESH:D018384',
  'reactive oxygen': 'MESH:D017382',
  'reactive oxygen intermediates': 'MESH:D017382',
  'reactive oxygen species': 'MESH:D017382',
  'reactive oxygen species assay': 'MESH:D017382',
  'reactive oxygen species formation': 'GO:GO:1903409',
  'reactive oxygen species generation': 'GO:GO:1903409',
  'reactive oxygen species levels': 'MESH:D017382',
  'reactive oxygen species production': 'GO:GO:1903409',
  'reactive oxygen specified': 'MESH:D017382',
  'reactive oxygen specious': 'MESH:D017382',
  'reactive oxygen speices': 'MESH:D017382',
  'reactive oxygen spices': 'MESH:D017382',
  'reactive oxygen stress': 'MESH:D018384',
  'reactive oxygen substances': 'MESH:D017382',
  'reactive oxygenspecies': 'MESH:D017382',
  'reactive species of oxygen': 'MESH:D017382',
  'reactive species oxygen': 'MESH:D017382',
  'reduced oxidative stress': 'ungrounded',
  'region of support': 'ungrounded',
  'regression on order statistically': 'ungrounded',
  'rehmannia glutinosa oligosaccharides': 'CHEBI:CHEBI:50699',
  'robot operating system': 'ungrounded',
  'rod outer segments': 'MESH:D012374',
  'rod outer segments membrane': 'MESH:D012374',
  'rod photoreceptor outer segments': 'MESH:D012374',
  'roscovitine': 'CHEBI:CHEBI:45307',
  'rosiglitazone': 'CHEBI:CHEBI:50122',
  'rosuvastatin': 'CHEBI:CHEBI:38545'},
 {'MESH:D018384': 'Oxidative Stress',
  'MESH:D012516': 'Osteosarcoma',
  'MESH:D017382': 'Reactive Oxygen Species',
  'GO:GO:1903409': 'reactive oxygen species biosynthetic process',
  'CHEBI:CHEBI:50699': 'oligosaccharide',
  'MESH:D012374': 'Rod Cell Outer Segment',
  'CHEBI:CHEBI:45307': 'seliciclib',
  'CHEBI:CHEBI:50122': 'rosiglitazone',
  'CHEBI:CHEBI:38545': 'rosuvastatin'},
 ['CHEBI:CHEBI:38545', 'CHEBI:CHEBI:50122', 'MESH:D012374', 'MESH:D017382']]

In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-03 04:54:18] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-03 05:12:10] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9970649903236918 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'MESH:D012374': 174,
  'MESH:D017382': 48327,
  'MESH:D012516': 8,
  'CHEBI:CHEBI:45307': 3,
  'CHEBI:CHEBI:50699': 3,
  'GO:GO:1903409': 40,
  'CHEBI:CHEBI:50122': 13,
  'ungrounded': 10,
  'CHEBI:CHEBI:38545': 11,
  'MESH:D018384': 46},
 'f1': {'mean': 0.997065, 'std': 0.00018},
 'precision': {'mean': 0.996229, 'std': 0.000219},
 'recall': {'mean': 0.99798, 'std': 0.000202},
 'CHEBI:CHEBI:45307': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D012516': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'CHEBI:CHEBI:50122': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'MESH:D012374': {'f1': {'mean': 0.889662, 'std': 0.009024},
  'pr': {'mean': 0.81042, 'std': 0.012861},
  'rc': {'mean': 0.986437, 'std': 0.016615}},
 'CHEBI:CHEBI:38545': {'f1': {'mean': 0.266667, 'std': 0.326599},
  '

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for ROS

Produces the disambiguations:
	Osteosarcoma	MESH:D012516
	Oxidative Stress	MESH:D018384
	Reactive Oxygen Species*	MESH:D017382
	Rod Cell Outer Segment*	MESH:D012374
	oligosaccharide	CHEBI:CHEBI:50699
	reactive oxygen species biosynthetic process	GO:GO:1903409
	rosiglitazone*	CHEBI:CHEBI:50122
	rosuvastatin*	CHEBI:CHEBI:38545
	seliciclib	CHEBI:CHEBI:45307

Class level metrics:
--------------------
Grounding                                   	Count	F1     
                     Reactive Oxygen Species*	48327	0.99789
                      Rod Cell Outer Segment*	  174	0.88966
                            Oxidative Stress	   46	0.11868
reactive oxygen species biosynthetic process	   40	    0.0
                               rosiglitazone*	   13	    0.0
                                rosuvastatin*	   11	0.26667
                                  Ungrounded	   10	    0.0
                                Osteosarcoma	    8	    0.0
                                  s

In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1